# Read in data

In [1]:
from datetime import timedelta
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

airports = [
    "KATL",
    "KCLT",
    "KDEN",
    "KDFW",
    "KJFK",
    "KMEM",
    "KMIA",
    "KORD",
    "KPHX",
    "KSEA",
]

In [2]:
DATA_DIRECTORY = Path("code execution development data/old")

## LAMP

In [3]:
def read_lamp(airport):
    lamp = pd.read_csv(
        DATA_DIRECTORY / airport / f"{airport}_lamp.csv.bz2",
        parse_dates=["timestamp", "forecast_timestamp"],
        dtype={"temperature": "int16", "wind_direction":"int16", "wind_gust":"int16", "cloud_ceiling":"float16", "visibility":"int16"}
    )
    return lamp

## TBFM

In [4]:
def read_tbfm(airport):
    tbfm = pd.read_csv(
        DATA_DIRECTORY / airport / f"{airport}_tbfm.csv.bz2",
        parse_dates=["timestamp", "scheduled_runway_estimated_time"]
    )
    return tbfm

## ETD

In [5]:
def read_etd(airport):
    etd = pd.read_csv(
        DATA_DIRECTORY / airport / f"{airport}_etd.csv.bz2",
        parse_dates=["departure_runway_estimated_time", "timestamp"]
    )
    return etd

## MFS

In [6]:
def read_mfs(airport):
    etd = pd.read_csv(
        DATA_DIRECTORY / airport / f"{airport}_mfs.csv.bz2"
    )
    return etd

# Initial Model, Linear Regression

In [7]:
submission_format = pd.read_csv(
    "code execution development data/test_labels.csv", parse_dates=["timestamp"]
)

## Insert Data

In [23]:
def filter_lamp(current_time):
    # weather data
    valid_lamp = lamp.loc[(lamp.timestamp <= current_time) 
                    & (lamp.timestamp > valid_time) 
                    & (lamp.forecast_timestamp <= current_time) 
                    & (lamp.forecast_timestamp > valid_time)]
    return valid_lamp.iloc[-1]
    
def filter_tbfm(current_time):
    # traffic data
    valid_tbfm = tbfm.loc[(tbfm.timestamp <= current_time) 
                    & (tbfm.timestamp > valid_time)]
    return valid_tbfm['gufi'].nunique()
    
def filter_etd(current_time):    
    #etd data
    valid_etd = etd.loc[(etd.timestamp <= current_time) 
                    & (etd.timestamp > valid_time) 
                    & (etd.gufi == df.loc[i].gufi)]
    try:
        return (valid_etd.iloc[-1].departure_runway_estimated_time - current_time).total_seconds()
    except:
        return 3600

In [22]:
full_frame = pd.DataFrame()
for a in airports:
    airport = a
    df = submission_format[submission_format.airport == a]
    times = df.timestamp.unique()

    etd = read_etd(airport)
    tbfm = read_tbfm(airport)
    lamp = read_lamp(airport)

    for t in times:
        indices = df[df.timestamp == t].index
        current_time = pd.to_datetime(t)
        valid_time = current_time - pd.Timedelta(30, unit='hours')
        
        # insert etd data
        for i in indices:
            df.loc[i, 'etd'] = filter_etd(current_time)
            
        # insert traffic data
        df.loc[indices, 'traffic'] = filter_tbfm(current_time)

        # insert weather data
        current_forecast = filter_lamp(current_time)
        df.loc[indices, 'precip'] = current_forecast.precip
        df.loc[indices, 'lightning_prob'] = current_forecast.lightning_prob
        df.loc[indices, 'cloud'] = filter_lamp(current_time).cloud
        df.loc[indices, 'visibility'] = current_forecast.visibility
        df.loc[indices, 'cloud_ceiling'] = current_forecast.cloud_ceiling
        df.loc[indices, 'wind_gust'] = current_forecast.wind_gust
        df.loc[indices, 'wind_speed'] = current_forecast.wind_speed
        df.loc[indices, 'wind_direction'] = current_forecast.wind_direction
        df.loc[indices, 'temperature'] = current_forecast.temperature
        
    full_frame = pd.concat([full_frame, df])
        
# insert metadata
metadata = pd.concat([read_mfs('KATL'),
                      read_mfs('KCLT'),
                      read_mfs('KDEN'),
                      read_mfs('KDFW'),
                      read_mfs('KJFK'),
                      read_mfs('KMEM'),
                      read_mfs('KMIA'),
                      read_mfs('KORD'),
                      read_mfs('KPHX'),
                      read_mfs('KSEA')])

full_frame = full_frame.merge(metadata, how='left', on='gufi')

full_frame.head(5)

C:\Users\Shantel\AppData\Local\Temp\ipykernel_28964\3381460652.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i, 'etd'] = filter_etd(current_time)
C:\Users\Shantel\AppData\Local\Temp\ipykernel_28964\3381460652.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[indices, 'traffic'] = filter_tbfm(current_time)
C:\Users\Shantel\AppData\Local\Temp\ipykernel_28964\3381460652.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

gufi                               157513
timestamp                          157513
scheduled_runway_estimated_time    157513
dtype: int64 833
gufi                               163299
timestamp                          163299
scheduled_runway_estimated_time    163299
dtype: int64 877
gufi                               156727
timestamp                          156727
scheduled_runway_estimated_time    156727
dtype: int64 808
gufi                               156657
timestamp                          156657
scheduled_runway_estimated_time    156657
dtype: int64 814
gufi                               215591
timestamp                          215591
scheduled_runway_estimated_time    215591
dtype: int64 1253
gufi                               216809
timestamp                          216809
scheduled_runway_estimated_time    216809
dtype: int64 1241
gufi                               215440
timestamp                          215440
scheduled_runway_estimated_time    215440
dtype: int64 1

C:\Users\Shantel\AppData\Local\Temp\ipykernel_28964\3381460652.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i, 'etd'] = filter_etd(current_time)


gufi                               169984
timestamp                          169984
scheduled_runway_estimated_time    169984
dtype: int64 882
gufi                               164995
timestamp                          164995
scheduled_runway_estimated_time    164995
dtype: int64 860


C:\Users\Shantel\AppData\Local\Temp\ipykernel_28964\3381460652.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[indices, 'traffic'] = filter_tbfm(current_time)
C:\Users\Shantel\AppData\Local\Temp\ipykernel_28964\3381460652.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[indices, 'precip'] = current_forecast.precip
C:\Users\Shantel\AppData\Local\Temp\ipykernel_28964\3381460652.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

...

gufi                               255325
timestamp                          255325
scheduled_runway_estimated_time    255325
dtype: int64 639
gufi                               253798
timestamp                          253798
scheduled_runway_estimated_time    253798
dtype: int64 629
gufi                               252384
timestamp                          252384
scheduled_runway_estimated_time    252384
dtype: int64 620
gufi                               201219
timestamp                          201219
scheduled_runway_estimated_time    201219
dtype: int64 496
gufi                               227027
timestamp                          227027
scheduled_runway_estimated_time    227027
dtype: int64 576
gufi                               218379
timestamp                          218379
scheduled_runway_estimated_time    218379
dtype: int64 553
gufi                               179744
timestamp                          179744
scheduled_runway_estimated_time    179744
dtype: int64 404

C:\Users\Shantel\AppData\Local\Temp\ipykernel_28964\3381460652.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i, 'etd'] = filter_etd(current_time)
C:\Users\Shantel\AppData\Local\Temp\ipykernel_28964\3381460652.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[indices, 'traffic'] = filter_tbfm(current_time)
C:\Users\Shantel\AppData\Local\Temp\ipykernel_28964\3381460652.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

gufi                               241723
timestamp                          241723
scheduled_runway_estimated_time    241723
dtype: int64 592
gufi                               232722
timestamp                          232722
scheduled_runway_estimated_time    232722
dtype: int64 567
gufi                               223216
timestamp                          223216
scheduled_runway_estimated_time    223216
dtype: int64 562
gufi                               252774
timestamp                          252774
scheduled_runway_estimated_time    252774
dtype: int64 644
gufi                               257336
timestamp                          257336
scheduled_runway_estimated_time    257336
dtype: int64 650
gufi                               252869
timestamp                          252869
scheduled_runway_estimated_time    252869
dtype: int64 640
gufi                               251525
timestamp                          251525
scheduled_runway_estimated_time    251525
dtype: int64 626

C:\Users\Shantel\AppData\Local\Temp\ipykernel_28964\2764470233.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  etd = pd.read_csv(
C:\Users\Shantel\AppData\Local\Temp\ipykernel_28964\2764470233.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  etd = pd.read_csv(


,gufi,timestamp,airport,minutes_until_pushback,etd,traffic,precip,lightning_prob,cloud,visibility,cloud_ceiling,wind_gust,wind_speed,wind_direction,temperature,aircraft_engine_class,aircraft_type,major_carrier,flight_type,isdeparture
0,AAL1227.ATL.MIA.201114.1242.0052.TFM,2020-11-15 11:15:00,KATL,86,6300.0,833.0,False,N,FW,7.0,8.0,0.0,2.0,13.0,54.0,JET,A319,AAL,SCHEDULED_AIR_TRANSPORT,True
1,AAL1227.ATL.MIA.201114.1242.0052.TFM,2020-11-15 11:15:00,KATL,86,6300.0,833.0,False,N,FW,7.0,8.0,0.0,2.0,13.0,54.0,JET,A319,AAL,SCHEDULED_AIR_TRANSPORT,False
2,AAL1227.ATL.MIA.201114.1242.0052.TFM,2020-11-15 12:00:00,KATL,41,3600.0,877.0,False,N,SC,7.0,8.0,0.0,5.0,14.0,53.0,JET,A319,AAL,SCHEDULED_AIR_TRANSPORT,True
3,AAL1227.ATL.MIA.201114.1242.0052.TFM,2020-11-15 12:00:00,KATL,41,3600.0,877.0,False,N,SC,7.0,8.0,0.0,5.0,14.0,53.0,JET,A319,AAL,SCHEDULED_AIR_TRANSPORT,False
4,AAL153.ATL.CLT.201114.1137.0016.TFM,2020-11-15 10:30:00,KATL,62,4980.0,808.0,False,N,SC,7.0,8.0,0.0,5.0,11.0,53.0,JET,A320,AAL,SCHEDULED_AIR_TRANSPORT,True


## Feature Engineering

In [34]:
full_frame.nunique()

gufi                                   1228
timestamp                                17
minutes_until_pushback                  133
etd                                     142
traffic                                 114
                                       ... 
aircraft_type_MD11                        2
aircraft_type_PC12                        2
major_carrier_DAL                         2
major_carrier_UAL                         2
flight_type_SCHEDULED_AIR_TRANSPORT       2
Length: 68, dtype: int64

In [32]:
# binary encoding
full_frame.replace(False, 0, inplace=True)
full_frame.replace(True, 0, inplace=True)

In [33]:
# nominal encoding
full_frame = pd.get_dummies(full_frame, columns=['airport',
                                 'cloud', 
                                 'lightning_prob',
                                 'aircraft_engine_class',
                                 'aircraft_type',
                                 'major_carrier',
                                 'flight_type'], 
                    drop_first=True)
full_frame.head(5)

KeyError: "None of [Index(['airport', 'cloud', 'lightning_prob', 'aircraft_engine_class',\n       'aircraft_type', 'major_carrier', 'flight_type'],\n      dtype='object')] are in the [columns]"

# Linear Regression

## Model 1

In [35]:
import statsmodels.api as sm

Y = full_frame['minutes_until_pushback']
X = full_frame.drop(columns=['gufi', 'timestamp', 'minutes_until_pushback'])
X = sm.add_constant(X)

model = sm.OLS(Y,X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     minutes_until_pushback   R-squared:                       0.581
Model:                                OLS   Adj. R-squared:                  0.570
Method:                     Least Squares   F-statistic:                     50.53
Date:                    Sat, 01 Apr 2023   Prob (F-statistic):               0.00
Time:                            17:47:45   Log-Likelihood:                -10227.
No. Observations:                    2356   AIC:                         2.058e+04
Df Residuals:                        2292   BIC:                         2.095e+04
Df Model:                              63                                         
Covariance Type:                nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
const                                  25.1745     13.242      1.901      0.057      -0.793      51.141
etd                                     0.0131      0.000     52.505      0.000       0.013       0.014
traffic                                 0.0041      0.006      0.647      0.517      -0.008       0.016
precip                              -1.474e-12   8.83e-12     -0.167      0.867   -1.88e-11    1.58e-11
visibility                              2.5664      0.885      2.900      0.004       0.831       4.302
cloud_ceiling                          -6.4138      1.307     -4.907      0.000      -8.977      -3.850
wind_gust                               0.1354      0.163      0.830      0.406      -0.184       0.455
wind_speed                             -0.4282      0.329     -1.300      0.194      -1.074       0.218
wind_direction                         -0.1162      0.126     -0.923      0.356      -0.363       0.131
temperature                             0.2821      0.179      1.578      0.115      -0.068       0.633
isdeparture                         -4.405e-14   1.56e-13     -0.283      0.777    -3.5e-13    2.62e-13
airport_KCLT                           -6.3505      2.521     -2.519      0.012     -11.294      -1.407
airport_KDEN                            9.5061      6.315      1.505      0.132      -2.878      21.891
airport_KDFW                            1.1239      3.967      0.283      0.777      -6.656       8.904
airport_KJFK                            6.3374      4.881      1.298      0.194      -3.234      15.909
airport_KMEM                           16.8457      6.725      2.505      0.012       3.658      30.034
airport_KMIA                           -0.1095      6.741     -0.016      0.987     -13.329      13.110
airport_KORD                           -4.6892      4.477     -1.047      0.295     -13.468       4.089
airport_KPHX                           -6.4235      4.765     -1.348      0.178     -15.768       2.921
airport_KSEA                           17.4765      4.740      3.687      0.000       8.182      26.771
cloud_CL                                4.4843      3.525      1.272      0.203      -2.428      11.397
cloud_FW                                3.0821      1.324      2.327      0.020       0.485       5.679
cloud_OV                              -17.9941      4.890     -3.680      0.000     -27.583      -8.405
cloud_SC                                5.0977      1.653      3.083      0.002       1.855       8.340
lightning_prob_M                       35.8383     11.089      3.232      0.001      14.094      57.583
lightning_prob_N                       -6.9442      6.257     -1.110      0.267     -19.214       5.325
aircraft_engine_class_T

In [36]:
# calculate MAE
from sklearn.metrics import mean_absolute_error as mae
ypred = model.predict()
MAE = mae(Y, ypred)
MAE

10.231197185342388

In [37]:
# check for multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor

# create VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculate VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
vif_data

C:\Users\Shantel\.conda\envs\myenv\lib\site-packages\statsmodels\regression\linear_model.py:1752: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


,feature,VIF
0,const,1164.245307
1,etd,1.108416
2,traffic,21.694123
3,precip,NaN
4,visibility,3.114533
...,...,...
61,aircraft_type_MD11,3.218903
62,aircraft_type_PC12,1.937598
63,major_carrier_DAL,2.733234
64,major_carrier_UAL,2.937714


## Model 2

In [38]:
# reduce features
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector

# re-define independent variables
X = full_frame.drop(columns=['gufi', 'timestamp', 'minutes_until_pushback'])

# linear regression estimator to be used in feature selection
reg = LinearRegression().fit(X, Y)

In [39]:
# reduce features using SequentialFeatureSelector
sfs = SequentialFeatureSelector(reg, n_features_to_select=20)
sfs.fit(X, Y)

SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select=20)

In [40]:
# return selected features
sfs.get_feature_names_out()

array(['etd', 'wind_gust', 'wind_direction', 'airport_KCLT',
       'airport_KJFK', 'airport_KORD', 'cloud_OV', 'aircraft_type_A21N',
       'aircraft_type_A320', 'aircraft_type_A333', 'aircraft_type_B752',
       'aircraft_type_B764', 'aircraft_type_B772', 'aircraft_type_CRJ7',
       'aircraft_type_CRJ9', 'aircraft_type_E145', 'aircraft_type_E170',
       'aircraft_type_E75L', 'aircraft_type_PC12',
       'flight_type_SCHEDULED_AIR_TRANSPORT'], dtype=object)

In [42]:
# re-fit model
X = full_frame[['etd', 'wind_gust', 'wind_direction', 'airport_KCLT',
       'airport_KJFK', 'airport_KORD', 'cloud_OV', 'aircraft_type_A21N',
       'aircraft_type_A320', 'aircraft_type_A333', 'aircraft_type_B752',
       'aircraft_type_B764', 'aircraft_type_B772', 'aircraft_type_CRJ7',
       'aircraft_type_CRJ9', 'aircraft_type_E145', 'aircraft_type_E170',
       'aircraft_type_E75L', 'aircraft_type_PC12',
       'flight_type_SCHEDULED_AIR_TRANSPORT']]
X = sm.add_constant(X)

model2 = sm.OLS(Y,X).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     minutes_until_pushback   R-squared:                       0.555
Model:                                OLS   Adj. R-squared:                  0.551
Method:                     Least Squares   F-statistic:                     145.7
Date:                    Sat, 01 Apr 2023   Prob (F-statistic):               0.00
Time:                            17:49:20   Log-Likelihood:                -10299.
No. Observations:                    2356   AIC:                         2.064e+04
Df Residuals:                        2335   BIC:                         2.076e+04
Df Model:                              20                                         
Covariance Type:                nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
const                                  14.8173      2.144      6.911      0.000      10.613      19.022
etd                                     0.0129      0.000     52.852      0.000       0.012       0.013
wind_gust                               0.0919      0.054      1.701      0.089      -0.014       0.198
wind_direction                         -0.2298      0.073     -3.152      0.002      -0.373      -0.087
airport_KCLT                           -6.9866      1.507     -4.637      0.000      -9.942      -4.032
airport_KJFK                           -4.0936      2.267     -1.806      0.071      -8.538       0.351
airport_KORD                          -13.2500      1.989     -6.662      0.000     -17.150      -9.350
cloud_OV                                1.9402      1.241      1.563      0.118      -0.494       4.374
aircraft_type_A21N                    -12.3662     11.309     -1.094      0.274     -34.543       9.810
aircraft_type_A320                     -3.2943      1.781     -1.850      0.065      -6.787       0.198
aircraft_type_A333                    -15.8188      9.663     -1.637      0.102     -34.768       3.131
aircraft_type_B752                      6.3332      2.365      2.677      0.007       1.695      10.972
aircraft_type_B764                     14.5395      5.099      2.851      0.004       4.540      24.539
aircraft_type_B772                     -5.6232      3.670     -1.532      0.126     -12.820       1.573
aircraft_type_CRJ7                     -5.1740      1.793     -2.885      0.004      -8.690      -1.658
aircraft_type_CRJ9                     -3.6846      1.611     -2.288      0.022      -6.843      -0.526
aircraft_type_E145                     -7.6748      2.546     -3.014      0.003     -12.668      -2.682
aircraft_type_E170                     -8.0362      2.147     -3.742      0.000     -12.247      -3.825
aircraft_type_E75L                     -7.0269      1.796     -3.912      0.000     -10.549      -3.505
aircraft_type_PC12                    -11.8829      8.026     -1.481      0.139     -27.622       3.856
flight_type_SCHEDULED_AIR_TRANSPORT    -9.4999      1.604     -5.924      0.000     -12.644      -6.355
==============================================================================
Omnibus:                     2249.693   Durbin-Watson:                   0.884
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           120549.989
Skew:                           4.493   Prob(JB):                         0.00
Kurtosis:                      36.871   Cond. No.                     1.09e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the 

In [43]:
# re-calculate MAE
from sklearn.metrics import mean_absolute_error as mae
ypred = model.predict()
MAE = mae(Y, ypred)
MAE

10.231197185342388

In [44]:
# re-check for multicollinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor

# create VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculate VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
vif_data

,feature,VIF
0,const,29.256322
1,etd,1.015750
2,wind_gust,2.412839
3,wind_direction,2.019255
4,airport_KCLT,1.112467
5,airport_KJFK,1.098348
6,airport_KORD,2.008760
7,cloud_OV,1.761591
8,aircraft_type_A21N,1.035147
9,aircraft_type_A320,1.060267
